In [ ]:
# %% [code]
import asyncio
import base64
import hashlib
import hmac
import json
import math
import time
import uuid
import urllib.parse
from datetime import datetime, timezone
import logging
import requests
import networkx as nx
import websockets  # pip install websockets
import matplotlib.pyplot as plt
import numpy as np

# -------------------------------
# Logging Setup
# -------------------------------
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# -------------------------------
# KrakenArbitrageBot Class
# -------------------------------
class KrakenArbitrageBot:
    def __init__(self,
                 api_key,
                 api_secret,
                 tradable_pairs,
                 starting_balance=100.0,
                 run_duration=300,
                 min_order_usdt=5.05,
                 fee_per_leg=0.0022,
                 position_fraction=0.01,
                 min_profit_cutoff=0.22):
        """
        Theory:
            Build a graph with cryptocurrencies as nodes with conversion rates as edge weights. 
            By finding negative cycles on the edges of this graph, we find arbitrage opportunities.
            Filter these cycles for the most profitable, and execute the most profitable cycle if it will be profitable (inc. fees).
            Since the graph is small, NetworkX is adequate. For large graphs a C++ implementaiton and/or faster algorithm would be best.
            Populate graph by tracking order book data with Kraken websockets, place orders with API.
            When demo loop terminates the bot closes all opened (non USDT) positions to USDT.

        Execution:
            The bot works! In deployment, the bot almost never finds profitable cycles due to the exchange buy/sell fees (~0.22%/leg).
            This was the expected result, though. 
            During periods of high volatility, or by using leveraged funds, opportunities may still be found, but I haven't studied this. 
            When demo deployment loop terminates it shows: 
                * balance over time
                * the conversion graphs with edges weighted by frequency
                * most used nodes/edges 
            
            For 'true' deployment, use cron or other linux task scheduler to call frequently using a server/VM, 
            or buy a computer and run this script for a long time in the background.  

        Parameters:
          api_key, api_secret: Your Kraken API credentials.
          tradable_pairs: List of strings (e.g. ['XBT/USDT', 'ETH/USDT', ...]).
          starting_balance: Starting balance in USDT.
          run_duration: Total run time in seconds.
          min_order_usdt: Minimum order USDT equivalent per leg.
          fee_per_leg: Fee rate per leg (e.g. 0.0022 = 0.22%).
          position_fraction: Fraction of account balance used per cycle.
          min_profit_cutoff: Minimum profit per leg (%) required to execute a cycle.
                              (Effective profit must be >= n_legs * min_profit_cutoff;
                               set to 0 for debugging.)

        Other Notes: 
            Overall I'm very happy with this project, even though it unfortunately doesn't appear to be profitable. 
            However, that was the expected result, so I am not disappointed. 
            This was an ambitious project only possible for me with extensive use of coding aides (Stack Overflow, ChatGPT, etc), 
            and I learned a lot! 
        """
        self.api_key = api_key
        self.api_secret = api_secret
        self.tradable_pairs = tradable_pairs
        self.starting_balance = starting_balance
        self.balance = starting_balance
        self.run_duration = run_duration
        self.min_order_usdt = min_order_usdt
        self.fee_per_leg = fee_per_leg
        self.position_fraction = position_fraction
        self.min_profit_cutoff = min_profit_cutoff  # per leg cutoff in percent

        self.KR_API_BASE = "https://api.kraken.com"
        self.KR_WS = "wss://ws.kraken.com"

        # Data containers:
        self.prices = {}         # keyed by canonical symbol, e.g. "xbtusdt"
        self.trade_records = []  # list of executed cycle records
        self.balance_history = []  # list of (timestamp, balance)
        self.cycle_counter = {}  # counts cycles executed (keyed by cycle tuple)
        self.pending_cycle_ids = {}  # mapping: cycle tuple -> list of order IDs

        # Build a set of canonical pairs that we expect.
        self.expected_pairs = {self.canonical(p) for p in tradable_pairs}
        # Build Kraken asset pair mapping.
        self.kraken_asset_pairs = self.get_kraken_asset_pairs()
        logging.info("Kraken asset pair mapping: %s", self.kraken_asset_pairs)

    @staticmethod
    def canonical(pair_str):
        """Convert a pair like 'XBT/USDT' to 'xbtusdt'."""
        return pair_str.replace("/", "").lower()

    def normalize_asset(self, asset):
        """Normalize Kraken asset codes (remove leading 'X' or 'Z' if appropriate)."""
        if len(asset) == 4 and (asset.startswith("X") or asset.startswith("Z")):
            return asset[1:]
        return asset

    # --- Kraken API Helpers ---
    def kraken_request(self, uri_path, data):
        url = self.KR_API_BASE + uri_path
        data['nonce'] = str(int(time.time() * 1000))
        postdata = urllib.parse.urlencode(data)
        message = (data['nonce'] + postdata).encode()
        sha256 = hashlib.sha256(message).digest()
        sig_msg = uri_path.encode() + sha256
        signature = hmac.new(base64.b64decode(self.api_secret), sig_msg, hashlib.sha512).digest()
        sig_digest = base64.b64encode(signature)
        headers = {
            "API-Key": self.api_key,
            "API-Sign": sig_digest.decode()
        }
        response = requests.post(url, headers=headers, data=data)
        return response.json()

    def query_order(self, trade_id):
        data = {"txid": trade_id}
        return self.kraken_request("/0/private/QueryOrders", data)

    def is_order_filled(self, trade_id):
        result = self.query_order(trade_id)
        if result.get("error"):
            logging.error("Error querying order %s: %s", trade_id, result["error"])
            return False
        order_info = result.get("result", {}).get(trade_id, {})
        return order_info.get("status", "") == "closed"

    def get_balance(self):
        """Query account balance via Kraken's Balance API."""
        response = self.kraken_request("/0/private/Balance", {})
        if response.get("error"):
            logging.error("Error querying balance: %s", response["error"])
            return {}
        return response.get("result", {})

    def close_all_positions(self):
        """
        Re-implemented close_all_positions:
        - Query the account balance.
        - For each asset that is not USDT (after normalization), if the balance > 0,
          attempt to find a tradable pair to convert that asset back to USDT and execute a sell order.
        """
        logging.info("Closing all positions...")
        balance_data = self.get_balance()
        if not balance_data:
            logging.error("No balance data available.")
            return
        for asset, amount_str in balance_data.items():
            try:
                amount = float(amount_str)
            except Exception as e:
                logging.error("Error parsing balance for asset %s: %s", asset, e)
                continue
            norm = self.normalize_asset(asset)
            if norm.upper() == "USDT":
                continue
            candidate = None
            for p in self.tradable_pairs:
                base, quote = p.split("/")
                if base.upper() == norm.upper() and "USDT" in quote.upper():
                    candidate = p
                    break
            if candidate is None:
                logging.error("No tradable pair found to convert %s to USDT.", asset)
                continue
            rate = self.prices.get(self.canonical(candidate), None)
            if rate is None:
                logging.error("No market rate available for pair %s", candidate)
                continue
            txid = self.execute_order_leg(candidate, "sell", amount)
            if txid is None:
                logging.error("Failed to close position for asset %s", asset)
            else:
                logging.info("Closed position for asset %s via pair %s (txid: %s)", asset, candidate, txid)
                for _ in range(10):
                    if self.is_order_filled(txid):
                        break
                    time.sleep(1)

    def get_kraken_asset_pairs(self):
        url = self.KR_API_BASE + "/0/public/AssetPairs"
        try:
            response = requests.get(url)
            data = response.json()
            mapping = {}
            for key, info in data.get("result", {}).items():
                wsname = info.get("wsname", "")
                if wsname:
                    canon = self.canonical(wsname.replace("/", ""))
                    mapping[canon] = info.get("altname", wsname.replace("/", ""))
            return mapping
        except Exception as e:
            logging.error("Error fetching Kraken asset pairs: %s", e)
            return {}

    # --- Price Streaming ---
    async def stream_prices(self, runtime=None):
        if runtime is None:
            runtime = self.run_duration
        sub_msg = {
            "event": "subscribe",
            "pair": self.tradable_pairs,
            "subscription": {"name": "ticker"}
        }
        logging.info("Connecting to Kraken WebSocket: %s", self.KR_WS)
        try:
            async with websockets.connect(self.KR_WS) as ws:
                await ws.send(json.dumps(sub_msg))
                start_time = time.time()
                while time.time() - start_time < runtime:
                    try:
                        message = await asyncio.wait_for(ws.recv(), timeout=10)
                        data = json.loads(message)
                        if isinstance(data, dict):
                            continue
                        if isinstance(data, list) and len(data) >= 4 and data[2] == "ticker":
                            pair = data[3]
                            canon_pair = self.canonical(pair)
                            ticker_data = data[1]
                            if "c" in ticker_data and isinstance(ticker_data["c"], list):
                                try:
                                    price = float(ticker_data["c"][0])
                                    self.prices[canon_pair] = price
                                    logging.info("Price update: %s = %f", canon_pair, price)
                                except Exception as e:
                                    logging.error("Error parsing ticker price: %s", e)
                    except asyncio.TimeoutError:
                        logging.warning("Kraken WebSocket timeout waiting for data.")
                    except Exception as e:
                        logging.error("Error in Kraken WebSocket stream: %s", e)
                logging.info("Kraken price streaming period ended.")
        except Exception as e:
            logging.error("Error connecting to Kraken WebSocket: %s", e)

    # --- Graph and Cycle Detection ---
    def build_graph(self):
        G = nx.DiGraph()
        for symbol, price in self.prices.items():
            if price is None or price <= 0:
                continue
            if symbol not in self.expected_pairs:
                continue
            base = symbol[:3]
            quote = symbol[3:]
            if not base or not quote:
                continue
            try:
                weight_forward = -math.log(price)
                weight_reverse = -math.log(1/price)
            except Exception as e:
                logging.error("Error computing weights for %s: %s", symbol, e)
                continue
            G.add_edge(base, quote, weight=weight_forward, rate=price)
            G.add_edge(quote, base, weight=weight_reverse, rate=1/price)
        return G

    def find_best_cycle(self, G):
        best_cycle = None
        best_profit = 0
        for cycle in nx.simple_cycles(G):
            if "usdt" not in cycle:
                continue
            # Rotate the cycle so that it starts with 'usdt'
            while cycle[0] != "usdt":
                cycle = cycle[1:] + [cycle[0]]
            if len(cycle) < 2:
                continue
            total_weight = 0
            cycle_edges = []
            for i in range(len(cycle)):
                src = cycle[i]
                dst = cycle[(i+1) % len(cycle)]
                if G.has_edge(src, dst):
                    edge_data = G.get_edge_data(src, dst)
                    total_weight += edge_data['weight']
                    cycle_edges.append((src, dst, edge_data['rate']))
                else:
                    total_weight = None
                    break
            if total_weight is None:
                continue
            conversion_product = math.exp(-total_weight)
            profit_percent = (conversion_product - 1) * 100
            # Compute effective conversion after fees:
            n_legs = len(cycle_edges)
            effective_conversion = conversion_product * ((1 - self.fee_per_leg) ** n_legs)
            effective_profit = (effective_conversion - 1) * 100
            # Only consider cycles that yield profit above the cutoff (unless cutoff==0)
            if self.min_profit_cutoff > 0 and effective_profit < (n_legs * self.min_profit_cutoff):
                continue
            if profit_percent > best_profit:
                best_profit = profit_percent
                best_cycle = {
                    'cycle_nodes': cycle + [cycle[0]],
                    'edges': cycle_edges,
                    'conversion_product': conversion_product,
                    'profit_percent': profit_percent,
                    'n_legs': n_legs,
                    'effective_profit': effective_profit
                }
        return best_cycle

    # --- Cycle Execution ---
    def execute_cycle(self, cycle_info):
        orders = []
        trade_amount = self.position_fraction * self.balance  # trade amount for the cycle (USDT)
        current_amount = trade_amount
        cycle_nodes = cycle_info['cycle_nodes']
        n = len(cycle_nodes)
        for i in range(n - 1):
            current_asset = cycle_nodes[i]
            next_asset = cycle_nodes[i+1]
            pair, side, market_rate = self.get_order_details(current_asset, next_asset)
            if pair is None:
                logging.error("Failed to get order details for leg %s->%s", current_asset, next_asset)
                return None, current_amount
            if side == "buy":
                if current_amount < self.min_order_usdt:
                    logging.error("Insufficient USDT (%.2f) for minimum order on leg %s->%s", current_amount, current_asset, next_asset)
                    return None, current_amount
                volume = current_amount / market_rate
            else:
                volume = current_amount
                if volume * market_rate < self.min_order_usdt:
                    volume = self.min_order_usdt / market_rate
                    if volume * market_rate > current_amount:
                        logging.error("Not enough asset value for minimum order on leg %s->%s", current_asset, next_asset)
                        return None, current_amount
            txid = self.execute_order_leg(pair, side, volume)
            if txid is None:
                logging.error("Order execution failed for leg %s->%s", current_asset, next_asset)
                return None, current_amount
            orders.append(txid)
            for _ in range(10):
                if self.is_order_filled(txid):
                    break
                time.sleep(1)
            else:
                logging.error("Order %s on leg %s->%s did not fill in time.", txid, current_asset, next_asset)
                return None, current_amount
            if side == "buy":
                current_amount = volume * (1 - self.fee_per_leg)
            else:
                current_amount = volume * market_rate * (1 - self.fee_per_leg)
            logging.info("After leg %s->%s, trade amount updated: %.6f", current_asset, next_asset, current_amount)
        return orders, current_amount

    def get_order_details(self, current_asset, next_asset):
        candidates = []
        for p in self.tradable_pairs:
            parts = p.split("/")
            if current_asset.upper() in parts and next_asset.upper() in parts:
                candidates.append(p)
        if not candidates:
            logging.error("No asset pair found for conversion %s->%s", current_asset, next_asset)
            return None, None, None
        for candidate in candidates:
            base, quote = candidate.split("/")
            if current_asset.upper() == quote.upper() and next_asset.upper() == base.upper():
                rate = self.prices.get(self.canonical(candidate), 1)
                return candidate, "buy", rate
            elif current_asset.upper() == base.upper() and next_asset.upper() == quote.upper():
                rate = self.prices.get(self.canonical(candidate), 1)
                return candidate, "sell", rate
        candidate = candidates[0]
        base, quote = candidate.split("/")
        side = "buy" if current_asset.upper() == quote.upper() else "sell"
        rate = self.prices.get(self.canonical(candidate), 1)
        return candidate, side, rate

    def execute_order_leg(self, pair, side, volume):
        pair_canon = self.canonical(pair)
        if pair_canon in self.kraken_asset_pairs:
            kraken_pair = self.kraken_asset_pairs[pair_canon]
        else:
            if "usdt" in pair_canon:
                alt = pair_canon.replace("usdt", "usd")
            elif "usd" in pair_canon:
                alt = pair_canon.replace("usd", "usdt")
            else:
                alt = None
            if alt and alt in self.kraken_asset_pairs:
                kraken_pair = self.kraken_asset_pairs[alt]
            else:
                logging.error("Asset pair %s not recognized by Kraken.", pair)
                return None
        order_data = {
            "pair": kraken_pair,
            "type": side,
            "ordertype": "market",
            "volume": str(volume)
        }
        response = self.kraken_request("/0/private/AddOrder", order_data)
        if response.get("error"):
            logging.error("Order execution failed for pair %s: %s", kraken_pair, response["error"])
            return None
        result = response.get("result", {})
        txid = result.get("txid", [None])[0]
        logging.info("Order executed on %s: side=%s, volume=%.6f, txid=%s", kraken_pair, side, volume, txid)
        return txid

    # --- Trading Loop ---
    def trade_loop(self, check_interval=5):
        start_time = time.time()
        logging.info("Trade loop started. Running for %d seconds.", self.run_duration)
        while time.time() - start_time < self.run_duration:
            try:
                if self.balance < self.starting_balance * 0.95:
                    logging.warning("Loss limit exceeded (balance: %.2f). Stopping trading.", self.balance)
                    break

                for cycle_key in list(self.pending_cycle_ids.keys()):
                    txids = self.pending_cycle_ids[cycle_key]
                    if all(self.is_order_filled(txid) for txid in txids):
                        logging.info("Cycle %s completed. Removing from pending list.", cycle_key)
                        del self.pending_cycle_ids[cycle_key]

                G = self.build_graph()
                logging.info("Graph rebuilt: %d nodes, %d edges", G.number_of_nodes(), G.number_of_edges())
                best_cycle = self.find_best_cycle(G)
                if best_cycle is None:
                    logging.info("No profitable arbitrage cycle detected at this time.")
                else:
                    n = best_cycle['n_legs']
                    effective_conversion = best_cycle['conversion_product'] * ((1 - self.fee_per_leg) ** n)
                    effective_profit = (effective_conversion - 1) * 100
                    required_cutoff = n * self.min_profit_cutoff
                    if self.min_profit_cutoff > 0 and effective_profit < required_cutoff:
                        logging.info("Cycle effective profit %.4f%% below cutoff %.4f%%. Skipping.", effective_profit, required_cutoff)
                    else:
                        if best_cycle['cycle_nodes'][0] != "usdt":
                            logging.info("Cycle does not start with USDT after rotation. Skipping.")
                        else:
                            cycle_key = tuple(best_cycle['cycle_nodes'])
                            if cycle_key in self.pending_cycle_ids:
                                logging.info("Cycle %s already pending. Skipping duplicate.", best_cycle['cycle_nodes'])
                            else:
                                logging.info("Executing cycle: %s, raw profit=%.4f%%, effective profit=%.4f%%", 
                                             best_cycle['cycle_nodes'], best_cycle['profit_percent'], effective_profit)
                                orders, final_trade_amount = self.execute_cycle(best_cycle)
                                if orders is not None and final_trade_amount > (self.position_fraction * self.balance):
                                    profit = final_trade_amount - (self.position_fraction * self.balance)
                                    self.balance += profit
                                    self.pending_cycle_ids[cycle_key] = orders
                                    record = {
                                        'timestamp': datetime.now(timezone.utc).isoformat(),
                                        'cycle': best_cycle['cycle_nodes'],
                                        'orders': orders,
                                        'raw_conversion': best_cycle['conversion_product'],
                                        'raw_profit_percent': best_cycle['profit_percent'],
                                        'effective_profit_percent': effective_profit,
                                        'trade_amount': self.position_fraction * self.balance,
                                        'final_trade_amount': final_trade_amount,
                                        'profit': profit,
                                        'balance_before': self.balance - profit,
                                        'balance_after': self.balance
                                    }
                                    self.trade_records.append(record)
                                    self.cycle_counter[cycle_key] = self.cycle_counter.get(cycle_key, 0) + 1
                                    logging.info("Cycle executed. Profit: %.6f USDT. New balance: %.2f USDT", profit, self.balance)
                                else:
                                    logging.error("Cycle execution failed or unprofitable. Initiating close all.")
                                    self.close_all_positions()
            except Exception as e:
                logging.error("Exception in trade loop iteration: %s", e)
            self.balance_history.append((datetime.now(timezone.utc), self.balance))
            time.sleep(check_interval)
        logging.info("Trade loop finished after %.2f seconds.", time.time() - start_time)
        self.close_all_positions()

    # --- Performance Metrics & Visualization ---
    def compute_trade_statistics(self):
        num_trades = len(self.trade_records)
        if num_trades == 0:
            return {}
        profits = [((rec['balance_after'] - rec['balance_before']) / rec['balance_before']) for rec in self.trade_records]
        wins = [p for p in profits if p > 0]
        losses = [p for p in profits if p <= 0]
        avg_win = np.mean(wins) if wins else 0
        avg_loss = np.mean(losses) if losses else 0
        win_rate = (len(wins) / num_trades) * 100
        total_profit = (self.balance - self.starting_balance) / self.starting_balance * 100
        max_dd = self.compute_max_drawdown(self.balance_history)
        most_freq = sorted(self.cycle_counter.items(), key=lambda x: x[1], reverse=True)
        return {
            'num_trades': num_trades,
            'total_profit_percent': total_profit,
            'max_drawdown_percent': max_dd,
            'win_rate_percent': win_rate,
            'average_win_percent': avg_win * 100,
            'average_loss_percent': avg_loss * 100,
            'most_frequent_cycles': most_freq
        }

    @staticmethod
    def compute_max_drawdown(balance_history):
        if not balance_history:
            return 0
        peak = balance_history[0][1]
        max_dd = 0
        for _, bal in balance_history:
            if bal > peak:
                peak = bal
            dd = (peak - bal) / peak
            if dd > max_dd:
                max_dd = dd
        return max_dd * 100

    def plot_balance_history(self):
        if not self.balance_history:
            logging.info("No balance history available.")
            return
        times, bals = zip(*self.balance_history)
        plt.figure(figsize=(10,6))
        plt.plot(times, bals, marker='o')
        plt.title("Balance Over Time (USDT)")
        plt.xlabel("Time (UTC)")
        plt.ylabel("Balance (USDT)")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    def visualize_full_graph(self):
        """Visualize the current arbitrage graph (based on current prices)."""
        G = self.build_graph()
        pos = nx.spring_layout(G)
        plt.figure(figsize=(10,8))
        nx.draw_networkx_nodes(G, pos, node_color="skyblue", node_size=800)
        nx.draw_networkx_edges(G, pos, arrowstyle='->', arrowsize=20)
        nx.draw_networkx_labels(G, pos, font_size=12, font_family="sans-serif")
        edge_labels = {(u, v): f"{G[u][v]['rate']:.2f}" for u, v in G.edges()}
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
        plt.title("Current Arbitrage Graph")
        plt.axis("off")
        plt.show()

    def visualize_edge_usage(self):
        """Aggregate and visualize edge usage from executed cycles."""
        edge_usage = {}
        for cycle_tuple, count in self.cycle_counter.items():
            for i in range(len(cycle_tuple)-1):
                edge = (cycle_tuple[i], cycle_tuple[i+1])
                edge_usage[edge] = edge_usage.get(edge, 0) + count
        G_usage = nx.DiGraph()
        for (u, v), weight in edge_usage.items():
            G_usage.add_edge(u, v, weight=weight)
        pos = nx.spring_layout(G_usage)
        plt.figure(figsize=(10,8))
        widths = [G_usage[u][v]['weight'] for u, v in G_usage.edges()]
        nx.draw_networkx_nodes(G_usage, pos, node_color="lightgreen", node_size=800)
        nx.draw_networkx_edges(G_usage, pos, arrowstyle='->', arrowsize=20, width=widths)
        nx.draw_networkx_labels(G_usage, pos, font_size=12, font_family="sans-serif")
        edge_labels = {(u,v): G_usage[u][v]['weight'] for u,v in G_usage.edges()}
        nx.draw_networkx_edge_labels(G_usage, pos, edge_labels=edge_labels)
        plt.title("Edge Usage in Executed Arbitrage Cycles")
        plt.axis("off")
        plt.show()

    def visualize_node_usage(self):
        """Aggregate and visualize node usage from executed cycles."""
        node_usage = {}
        for cycle_tuple, count in self.cycle_counter.items():
            for node in cycle_tuple:
                node_usage[node] = node_usage.get(node, 0) + count
        G_nodes = nx.DiGraph()
        for node, weight in node_usage.items():
            G_nodes.add_node(node, weight=weight)
        pos = nx.spring_layout(G_nodes)
        sizes = [G_nodes.nodes[node]['weight'] * 200 for node in G_nodes.nodes()]
        plt.figure(figsize=(8,6))
        nx.draw_networkx_nodes(G_nodes, pos, node_color="orange", node_size=sizes)
        nx.draw_networkx_labels(G_nodes, pos, font_size=12)
        plt.title("Node Usage in Executed Arbitrage Cycles")
        plt.axis("off")
        plt.show()

    def performance_summary(self):
        """Print performance metrics and show visualizations."""
        stats = self.compute_trade_statistics()
        logging.info("Performance Summary:")
        for k, v in stats.items():
            logging.info("%s: %s", k, v)
        self.plot_balance_history()
        self.visualize_full_graph()
        self.visualize_edge_usage()
        self.visualize_node_usage()

    # --- Main Runner ---
    async def run(self):
        try:
            trade_task = asyncio.to_thread(self.trade_loop, check_interval=3) # or 5 - number of seconds between checks
            stream_task = self.stream_prices(runtime=self.run_duration)
            await asyncio.gather(trade_task, stream_task)
        finally:
            logging.info("Demo time ended or terminated. Closing positions...")
            self.close_all_positions()
            stats = self.compute_trade_statistics()
            logging.info("Live Demo Performance Metrics:")
            for k, v in stats.items():
                logging.info("%s: %s", k, v)
            self.plot_balance_history()
            # Optionally show graph visualizations:
            self.visualize_full_graph()
            self.visualize_edge_usage()
            self.visualize_node_usage()



# TRADABLE_PAIRS = [
#                     'USDT/USD', 'USDC/USD', 'USDC/USDT',                          # USDC, USDT, USD (test)
#                     'XBT/USDT', 'XBT/USDC', 'XBT/USD',                            # BTC
#                     'ETH/USDT', 'ETH/XBT', 'ETH/USDC', 'ETH/USD',                 # ETH
#                     'XRP/USDT', 'XRP/XBT', 'XRP/ETH', 'XRP/USDC', 'XRP/USD',      # XRP
#                     'SOL/USDT', 'SOL/ETH', 'SOL/XBT', 'SOL/USDC', 'SOL/USD',      # SOL
#                     'ADA/USDT', 'ADA/XBT', 'ADA/ETH', 'ADA/USDC', 'ADA/USD',      # ADA
#                     'DOGE/USDT', 'DOGE/XBT', 'DOGE/USDC', 'DOGE/USD',             # DOGE
#                     'LTC/USDT', 'LTC/XBT', 'LTC/ETH', 'LTC/USDC', 'LTC/USD',      # LTC
#                     'DOT/USDT', 'DOT/XBT', 'DOT/ETH', 'DOT/USDC', 'DOT/USD',      # DOT
#                     'LINK/USDT', 'LINK/XBT', 'LINK/ETH', 'LINK/USDC', 'LINK/USD', # LINK
#                     'XMR/USDT', 'XMR/USDC', 'XMR/USD', 'XMR/XBT',                 # XMR (test)
#                     'ETC/USD', 'ETC/ETH', 'ETC/XBT',                              # ETC (test)
#                     'BCH/USD', 'BCH/USDT', 'BCH/USD', 'BCH/XBT', 'BCH/ETH',       # BCH (test)
#                     ]


# # -------------------------------
# # Run the Bot in Jupyter Notebook
# # -------------------------------
# try:
#     bot = KrakenArbitrageBot(
#         api_key=KR_API_KEY,
#         api_secret=KR_API_SECRET,
#         tradable_pairs=TRADABLE_PAIRS,
#         starting_balance=100.0,
#         run_duration=60,
#         min_order_usdt=5.05,
#         fee_per_leg=0.0022,
#         position_fraction=1,
#         min_profit_cutoff=0.22  # set to 0 for debugging
#     )
#     # For Jupyter Notebook, apply nest_asyncio:
#     import nest_asyncio
#     nest_asyncio.apply()
#     asyncio.run(bot.run())
# except Exception as e:
#     logging.error("Error in live demo: %s", e)